In [1]:
import cyecca.util
from cyecca import lie
import casadi as ca
import numpy as np
import cyecca

In [2]:
a = ca.SX.sym("a", 3)
omega = ca.SX.sym("omega", 3)

std_vel = ca.SX.sym("std_vel")
std_acc = ca.SX.sym("std_acc")
std_rot = ca.SX.sym("std_rot")

W = ca.SX.sym("W", ca.Sparsity_lower(9))
nu = lie.se23.elem(ca.vertcat(a, ca.SX.zeros(3, 1), omega))

Q = ca.SX.sym("Q", ca.Sparsity_diag(9))
Q = ca.diagcat(
    std_vel**2,
    std_vel**2,
    std_vel**2,
    std_acc**2,
    std_acc**2,
    std_acc**2,
    std_rot**2,
    std_rot**2,
    std_rot**2,
)
F = nu.ad()
B = ca.SX.eye(9)
W_dot_lt = ca.tril(cyecca.util.sqrt_covariance_predict(W, F, Q))
f_W_dot_lt = ca.Function(
    "f_cov_predict",
    [W, a, omega, std_vel, std_acc, std_rot],
    [W_dot_lt],
    ["W", "a", "omega", "std_vel", "std_acc", "std_rot"],
    ["W_dot_lt"],
)

dt = ca.SX.sym("dt")
W1 = cyecca.util.rk4(
    lambda t, y: f_W_dot_lt(W, a, omega, std_vel, std_acc, std_rot), 0, W, dt
)

In [3]:
Xh = lie.SE23Mrp.elem(ca.SX.sym("Xh", 9))
y_mag = lie.r3.elem(ca.SX.sym("y_mag", 3))
B_w = lie.r3.elem(ca.SX.sym("B_w", 3))


y_mag_hat = Xh.R.inverse() @ B_w
e_mag = y_mag - y_mag_hat

lie.SO3Mrp.from_Euler(lie.SO3EulerB321.elem(ca.vertcat(1, 0, 0)))

SO3MrpLieGroup: SX(@1=0, [@1, @1, 0.255342])

In [4]:
mag_incl = ca.SX.sym("mag_incl")
mag_decl = ca.SX.sym("mag_decl")

# using ENU
east_hat = lie.r3.elem(ca.vertcat(1, 0, 0))
north_hat = lie.r3.elem(ca.vertcat(0, 1, 0))
up_hat = lie.r3.elem(ca.vertcat(0, 0, 1))

b_w = lie.SO3EulerB321.elem(ca.vertcat(-mag_decl, 0, -mag_incl)) @ north_hat
cyecca.symbolic.casadi_to_sympy(b_w.param)

Matrix([
[sin(mag_decl)*cos(mag_incl)],
[cos(mag_decl)*cos(mag_incl)],
[             -sin(mag_incl)]])

In [5]:
y_mag_w = Xh.R.inverse() @ y_mag
psi_m = ca.atan2(y_mag_w.param[0], y_mag_w.param[1])
psi_m

SX(@1=1, @2=8, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=sq((@1+@3)), @5=(4*(@1-@3)), atan2(((((@1-((@2*(sq(Xh_8)+sq(Xh_7)))/@4))*y_mag_0)+((((@2*(Xh_6*Xh_7))+(@5*Xh_8))/@4)*y_mag_1))+((((@2*(Xh_6*Xh_8))-(@5*Xh_7))/@4)*y_mag_2)),((((((@2*(Xh_7*Xh_6))-(@5*Xh_8))/@4)*y_mag_0)+((@1-((@2*(sq(Xh_8)+sq(Xh_6)))/@4))*y_mag_1))+((((@2*(Xh_7*Xh_8))+(@5*Xh_6))/@4)*y_mag_2))))

In [6]:
psi_hat = ca.atan2(b_w.param[0], b_w.param[1])
import sympy

sympy.simplify(
    cyecca.symbolic.casadi_to_sympy(psi_hat)
)  # just mag_decl, but should use this to figure out uncertainty?

atan2(sin(mag_decl)*cos(mag_incl), cos(mag_decl)*cos(mag_incl))

In [7]:
psi_m - mag_decl

SX(@1=1, @2=8, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=sq((@1+@3)), @5=(4*(@1-@3)), (atan2(((((@1-((@2*(sq(Xh_8)+sq(Xh_7)))/@4))*y_mag_0)+((((@2*(Xh_6*Xh_7))+(@5*Xh_8))/@4)*y_mag_1))+((((@2*(Xh_6*Xh_8))-(@5*Xh_7))/@4)*y_mag_2)),((((((@2*(Xh_7*Xh_6))-(@5*Xh_8))/@4)*y_mag_0)+((@1-((@2*(sq(Xh_8)+sq(Xh_6)))/@4))*y_mag_1))+((((@2*(Xh_7*Xh_8))+(@5*Xh_6))/@4)*y_mag_2)))-mag_decl))

In [8]:
crz = lie.SO3EulerB321.elem(ca.vertcat(psi_m - mag_decl, 0, 0)).log().param[2]
f_crz = ca.Function("f_crz", [Xh.param, y_mag.param, mag_decl], [crz])
f_crz([0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], 0)

DM(0)

In [9]:
crz

SX(@1=2, @2=0, @3=1, @4=8, @5=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @6=sq((@3+@5)), @7=4, @8=(@7*(@3-@5)), @9=(atan2(((((@3-((@4*(sq(Xh_8)+sq(Xh_7)))/@6))*y_mag_0)+((((@4*(Xh_6*Xh_7))+(@8*Xh_8))/@6)*y_mag_1))+((((@4*(Xh_6*Xh_8))-(@8*Xh_7))/@6)*y_mag_2)),((((((@4*(Xh_7*Xh_6))-(@8*Xh_8))/@6)*y_mag_0)+((@3-((@4*(sq(Xh_8)+sq(Xh_6)))/@6))*y_mag_1))+((((@4*(Xh_7*Xh_8))+(@8*Xh_6))/@6)*y_mag_2)))-mag_decl), @10=cos(@9), @11=cos(@9), @12=(@2<((@10+@11)+@3)), @13=(!@12), @14=(@2&&(@3<@10)), @15=(!@14), @16=(@3<@11), @17=nan, @18=(@13?(@14?(@13?(@15?(@16?@17:0):0):0):0):0), @19=0.5, @20=(@19*sqrt((((@3+@10)+@11)+@3))), @21=(!@16), @22=sin(@9), @23=sin(@9), @24=(@19*sqrt((((@3-@10)-@11)+@3))), @25=((@12?@20:0)+(@13?((@14?@17:0)+(@15?((@16?@17:0)+(@21?((@22+@23)/(@7*@24)):0)):0)):0)), @26=((@12?((@22+@23)/(@7*@20)):0)+(@13?((@14?@17:0)+(@15?((@16?@17:0)+(@21?@24:0)):0)):0)), @27=(@25*@26), @28=sq(@25), @29=(@13?(@15?(@16?(@13?(@15?(@16?@17:0):0):0):0):0):0), @30=(@13?(@14?(@13?(@14?@17:0):0):0):0), @31=s

In [10]:
f_crz(Xh.param, y_mag.param, mag_decl)

SX(@1=2, @2=0, @3=1, @4=8, @5=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @6=sq((@3+@5)), @7=4, @8=(@7*(@3-@5)), @9=(atan2(((((@3-((@4*(sq(Xh_8)+sq(Xh_7)))/@6))*y_mag_0)+((((@4*(Xh_6*Xh_7))+(@8*Xh_8))/@6)*y_mag_1))+((((@4*(Xh_6*Xh_8))-(@8*Xh_7))/@6)*y_mag_2)),((((((@4*(Xh_7*Xh_6))-(@8*Xh_8))/@6)*y_mag_0)+((@3-((@4*(sq(Xh_8)+sq(Xh_6)))/@6))*y_mag_1))+((((@4*(Xh_7*Xh_8))+(@8*Xh_6))/@6)*y_mag_2)))-mag_decl), @10=cos(@9), @11=cos(@9), @12=(@2<((@10+@11)+@3)), @13=(!@12), @14=(@2&&(@3<@10)), @15=(!@14), @16=(@3<@11), @17=nan, @18=(@13?(@14?(@13?(@15?(@16?@17:0):0):0):0):0), @19=0.5, @20=(@19*sqrt((((@3+@10)+@11)+@3))), @21=(!@16), @22=sin(@9), @23=sin(@9), @24=(@19*sqrt((((@3-@10)-@11)+@3))), @25=((@12?@20:0)+(@13?((@14?@17:0)+(@15?((@16?@17:0)+(@21?((@22+@23)/(@7*@24)):0)):0)):0)), @26=((@12?((@22+@23)/(@7*@20)):0)+(@13?((@14?@17:0)+(@15?((@16?@17:0)+(@21?@24:0)):0)):0)), @27=(@25*@26), @28=sq(@25), @29=(@13?(@15?(@16?(@13?(@15?(@16?@17:0):0):0):0):0):0), @30=(@13?(@14?(@13?(@14?@17:0):0):0):0), @31=s

In [11]:
psi_h = lie.SO3EulerB321.from_Mrp(Xh.R).param[0]
psi_h

SX(@1=8, @2=1, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=(4*(@2-@3)), @5=sq((@2+@3)), @6=asin((-(((@1*(Xh_8*Xh_6))-(@4*Xh_7))/@5))), @7=1.5708, @8=0.001, @9=(fabs((@6-@7))<@8), @10=(((@1*(Xh_7*Xh_8))-(@4*Xh_6))/@5), @11=(((@1*(Xh_6*Xh_8))+(@4*Xh_7))/@5), @12=(fabs((@6+@7))<@8), ((@9?atan2(@10,@11):0)+((!@9)?((@12?atan2((-@10),(-@11)):0)+((!@12)?atan2((((@1*(Xh_7*Xh_6))+(@4*Xh_8))/@5),(@2-((@1*(sq(Xh_8)+sq(Xh_7)))/@5))):0)):0)))

In [12]:
lie.SO3Mrp.from_Euler(lie.SO3EulerB321.elem(ca.vertcat(1, 0, 0)))

SO3MrpLieGroup: SX(@1=0, [@1, @1, 0.255342])

In [13]:
f_F = ca.Function("F", [a, omega, dt], [(nu * dt).exp(lie.SE23Mrp).Ad()])
F = f_F(a, omega, dt)

x_00 = ca.SX.sym("x_00", 9, 1)

x_10 = F @ x_00

np.array(f_F([1, 2, 3], [1, 2, 3], 0.1))

array([[ 0.9357548 , -0.28316496,  0.21019171,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.30293271,  0.95058062, -0.06803132,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.18054008,  0.12733457,  0.97529031,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.9357548 , -0.28316496,
         0.21019171, -0.12698783, -0.25970727,  0.21546746],
       [ 0.        ,  0.        ,  0.        ,  0.30293271,  0.95058062,
        -0.06803132,  0.29878045, -0.09768295, -0.03447152],
       [ 0.        ,  0.        ,  0.        , -0.18054008,  0.12733457,
         0.97529031, -0.15685769,  0.15169105, -0.04884147],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.9357548 , -0.28316496,  0.21019171],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0

In [14]:
f_F(a, omega, dt)

SX(@1=1, @2=8, @3=(dt*omega_0), @4=(dt*omega_1), @5=(dt*omega_2), @6=((sq(@3)+sq(@4))+sq(@5)), @7=sqrt(@6), @8=1e-07, @9=(fabs(@7)<@8), @10=0.5, @11=((@9?((@10+(-0.0416667*@6))+(0.00138889*sq(@6))):0)+((!@9)?((@1-cos(@7))/@6):0)), @12=(sq(@5)+sq(@4)), @13=(fabs(@7)<@8), @14=((@13?((0.166667+(-0.00833333*@6))+(0.000198413*sq(@6))):0)+((!@13)?((@7-sin(@7))/(@7*@6)):0)), @15=(@4*@3), @16=(@5*@3), @17=((@1-(@11*@12))+(@14*((@15*@5)-(@16*@4)))), @18=(sq(@5)+sq(@3)), @19=(@5*@4), @20=(@3*@4), @21=((@1-(@11*@18))+(@14*((@19*@3)-(@20*@5)))), @22=(sq(@4)+sq(@3)), @23=(@3*@5), @24=(@4*@5), @25=((@1-(@11*@22))+(@14*((@23*@4)-(@24*@3)))), @26=(0<((@17+@21)+@25)), @27=((@3+(@11*@24))-(@14*((@23*@5)+(@22*@3)))), @28=(((@11*@19)-@3)+(@14*((@20*@4)+(@18*@3)))), @29=4, @30=(@10*sqrt((((@1+@17)+@21)+@25))), @31=(!@26), @32=((@21<@17)&&(@25<@17)), @33=(@10*sqrt((((@1+@17)-@21)-@25))), @34=(!@32), @35=(@25<@21), @36=(((@11*@15)-@5)+(@14*((@12*@5)+(@16*@3)))), @37=((@5+(@11*@20))-(@14*((@18*@5)+(@19*@4))))

# Covariance Prediction

In [15]:
def derive_cov_sqrt_predict():
    a = ca.SX.sym("a", 3)
    omega = ca.SX.sym("omega", 3)

    std_vel = ca.SX.sym("std_vel")
    std_acc = ca.SX.sym("std_acc")
    std_rot = ca.SX.sym("std_rot")

    W = ca.SX.sym("W", ca.Sparsity_lower(9))
    nu = lie.se23.elem(ca.vertcat(a, ca.SX.zeros(3, 1), omega))

    Q = ca.SX.sym("Q", ca.Sparsity_diag(9))
    Q = ca.diagcat(
        std_vel**2,
        std_vel**2,
        std_vel**2,
        std_acc**2,
        std_acc**2,
        std_acc**2,
        std_rot**2,
        std_rot**2,
        std_rot**2,
    )
    F = nu.ad()
    B = ca.SX.eye(9)
    W_dot_lt = ca.tril(cyecca.util.sqrt_covariance_predict(W, F, Q))
    f_W_dot_lt = ca.Function(
        "f_cov_predict",
        [W, a, omega, std_vel, std_acc, std_rot],
        [W_dot_lt],
        ["W", "a", "omega", "std_vel", "std_acc", "std_rot"],
        ["W_dot_lt"],
    )

    dt = ca.SX.sym("dt")
    W1 = cyecca.util.rk4(
        lambda t, y: f_W_dot_lt(W, a, omega, std_vel, std_acc, std_rot), 0, W, dt
    )

    return ca.Function(
        "cov_sqrt_predict",
        [W, a, omega, std_vel, std_acc, std_rot, dt],
        [W1],
        ["W", "a", "omega", "std_vel", "std_acc", "std_rot", "dt"],
        ["W1"],
    )


f_cov_sqrt_predict = derive_cov_sqrt_predict()

In [16]:
def do_cov_sim():
    np.set_printoptions(precision=5, linewidth=200, suppress=True)
    W = 1 * np.eye(9)
    a = np.array([1, 2, 3])
    omega = np.array([1, 2, 3])
    dt = 0.01
    std_vel = 1
    std_acc = 2
    std_rot = 3
    for i in range(100):
        W = np.array(f_cov_sqrt_predict(W, a, omega, std_vel, std_acc, std_rot, dt))

    return W


do_cov_sim()

array([[ 1.41483,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  1.41483,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  1.41483,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  7.58644,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , -1.06175,  6.6539 ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , -1.59375, -3.89601,  2.73283,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , -0.00012,  2.51248, -0.49401,  1.88238,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , -2.20385, -0.35123,  0.24918,  0.53239,  2.18226,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  1.46866, -0.60115, -0.00184,  0.79898,  1.18259,  2.3485 ]])